In [1]:
import openai
import json
import requests
from typing import List, Dict, Any
import dotenv

dotenv.load_dotenv()

client = openai.OpenAI()
messages = []
BASE_URL = "https://nomad-movies.nomadcoders.workers.dev"

In [2]:
# --- 1. Tool Functions (Logic) ---


def get_popular_movies() -> str:
    """Fetch a list of currently popular movies."""
    try:
        response = requests.get(f"{BASE_URL}/movies", timeout=10)
        response.raise_for_status()
        return json.dumps(response.json()[:10])
    except Exception as e:
        return json.dumps({"error": str(e)})


def get_movie_details(movie_id: int) -> str:
    """Fetch detailed information about a specific movie by its ID."""
    try:
        response = requests.get(f"{BASE_URL}/movies/{movie_id}", timeout=10)
        response.raise_for_status()
        return json.dumps(response.json())
    except Exception as e:
        return json.dumps({"error": f"Movie ID {movie_id} not found."})


def get_similar_movies(movie_id: int) -> str:
    """Fetch a list of movies similar to a specific movie ID."""
    try:
        response = requests.get(f"{BASE_URL}/movies/{movie_id}/similar", timeout=10)
        response.raise_for_status()
        return json.dumps(response.json()[:5])
    except Exception as e:
        return json.dumps({"error": f"Similar movies for {movie_id} not found."})

In [ ]:
# --- 2. Configuration ---

FUNCTION_MAP = {
    "get_popular_movies": get_popular_movies,
    "get_movie_details": get_movie_details,
    "get_similar_movies": get_similar_movies,
}

TOOLS = [
    {
        "type": "function",
        "function": {
            "name": "get_popular_movies",
            "description": "Get a list of popular movies currently in theaters.",
            "parameters": {"type": "object", "properties": {}},
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_movie_details",
            "description": "Get detailed info for a movie. Requires a movie_id.",
            "parameters": {
                "type": "object",
                "properties": {
                    "movie_id": {
                        "type": "integer",
                        "description": "The ID of the movie.",
                    }
                },
                "required": ["movie_id"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_similar_movies",
            "description": "Get movies similar to a given movie ID.",
            "parameters": {
                "type": "object",
                "properties": {
                    "movie_id": {
                        "type": "integer",
                        "description": "The ID of the movie.",
                    }
                },
                "required": ["movie_id"],
            },
        },
    },
]

In [ ]:
# --- 3. Agent Logic ---


def process_ai_response(message):
    if message.tool_calls:
        # 1. Record assistant's tool call request
        messages.append(
            {
                "role": "assistant",
                "content": message.content or "",
                "tool_calls": [
                    {
                        "id": tc.id,
                        "type": "function",
                        "function": {
                            "name": tc.function.name,
                            "arguments": tc.function.arguments,
                        },
                    }
                    for tc in message.tool_calls
                ],
            }
        )

        # 2. Execute tools
        for tool_call in message.tool_calls:
            fname = tool_call.function.name
            args = json.loads(tool_call.function.arguments)

            print(f"[*] Calling Tool: {fname}({args})")

            func = FUNCTION_MAP.get(fname)
            result = func(**args)

            # 3. Append tool result back to history
            messages.append(
                {
                    "role": "tool",
                    "tool_call_id": tool_call.id,
                    "name": fname,
                    "content": result,
                }
            )

        # 4. Recursively call AI to handle results
        return call_ai()
    else:
        # Final response
        content = message.content
        messages.append({"role": "assistant", "content": content})
        print(f"AI: {content}")


def call_ai():
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        tools=TOOLS,
    )
    process_ai_response(response.choices[0].message)


# --- 4. Loop ---

print("--- Movie Agent Active (type 'quit' to exit) ---")
while True:
    user_input = input("You: ")
    if user_input.lower() == "quit":
        break

    messages.append({"role": "user", "content": user_input})
    call_ai()

--- Movie Agent Active (type 'quit' to exit) ---
[*] Calling Tool: get_popular_movies({})
AI: 현재 인기 있는 영화 목록은 다음과 같습니다:

1. **Shelter**
   - 개요: 외딴 섬에서 자발적인 은둔 생활을 하는 남자가 폭풍으로부터 소녀를 구하며 과거의 적들로부터 그녀를 보호하기 위해 은둔에서 나서게 되는 이야기입니다.
   - 개봉일: 2026-01-28
   - 평점: 6.8
   - ![포스터](https://image.tmdb.org/t/p/w780/buPFnHZ3xQy6vZEHxbHgL1Pc6CR.jpg)

2. **The Bluff**
   - 개요: 조용한 섬 생활이 복수심에 불타는 옛 선장의 귀환으로 흔들리며, 숙련된 전 해적이 가족을 지키기 위해 과거와 맞서 싸우는 이야기입니다.
   - 개봉일: 2026-02-17
   - 평점: 5.995
   - ![포스터](https://image.tmdb.org/t/p/w780/sojEzvfxR2DBcDSJyAisX8TWjov.jpg)

3. **Mercy**
   - 개요: 미래, 한 형사가 아내를 살해한 혐의로 재판 중이며, AI 판사에게 자신의 무죄를 입증하기 위해 90분 안에 싸워야 하는 이야기입니다.
   - 개봉일: 2026-01-20
   - 평점: 7.149
   - ![포스터](https://image.tmdb.org/t/p/w780/pyok1kZJCfyuFapYXzHcy7BLlQa.jpg)

4. **Pose**
   - 개요: 고독한 예술가가 그의 전 연인과 열정적이고 편집증적인 주말을 보내면서 혼란스러운 상황 속에서 새로운 예술적 영감을 얻으려는 이야기입니다.
   - 개봉일: 2026-02-25
   - 평점: 8.0
   - ![포스터](https://image.tmdb.org/t/p/w780/qhyyyWrHUbl6QG4udAJj17CBa5.jpg)

5. **Hellfire**
   - 개요